In [6]:
import cv2
import os
import numpy as np
from hmmlearn import hmm

# Function to load and preprocess images
def load_and_preprocess_images(image_directory, width, height):
    image_data = []
    for subdirectory in os.listdir(image_directory):
        subdirectory_path = os.path.join(image_directory, subdirectory)
        if os.path.isdir(subdirectory_path):
            for filename in os.listdir(subdirectory_path):
                image_path = os.path.join(subdirectory_path, filename)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if image is not None and not image.size == 0:
                    image = cv2.resize(image, (width, height))
                    image_data.append(image)

    return image_data

# Load and preprocess training data
train_directory = r"C:/NewML/MachineLearning/env/archive/weapon_detection/train/images"
image_width = 100
image_height = 100
training_data = load_and_preprocess_images(train_directory, image_width, image_height)

# Flatten and convert training_data to a 2D array
training_data = np.array(training_data)

# Labels for the training data (1 for weapon, 0 for no weapon)
num_weapon_images = len(training_data) // 2
num_no_weapon_images = len(training_data) // 2
training_labels = [1] * num_weapon_images + [0] * num_no_weapon_images
print("Number of training images:", len(training_data))
print("Shape of training_data:", np.shape(training_data))
# Flatten each image and stack them together
flattened_training_data = training_data.reshape(657, -1)

# Create and train the HMM model
model = hmm.MultinomialHMM(n_components=2)
model.fit(flattened_training_data)  # Make sure to use the correct training data

# Now, the shape of flattened_training_data should be (657, 10000), assuming 100x100 images


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Number of training images: 657
Shape of training_data: (657, 100, 100)


MultinomialHMM(n_components=2,
               n_trials=array([ 747545,  934610, 1020848, 1756172, 2376858, 1345830, 1199129,
       1400970, 1518311, 2296700, 2295516, 2227169, 2227510, 2500418,
       1985929, 2296380, 2098935, 2006062, 1682003,  595508, 2342017,
       2349948, 2313718, 2113846, 1105211, 2365537, 2357771, 1521670,
       1955037, 1778853, 2197483, 1313111, 1345968, 1938412, 1066624,
       1131000, 1829276, 1870446, 188...
       2421140, 2445104, 1744409,  460785, 2229859, 2350580, 2492207,
       1747657, 2321001,  317840, 2448652,  214709, 1409333, 2422340,
       2411525, 2388825, 1582526,  643327, 2486921,  707892, 2430859,
       2181478,  868920, 2492207, 2422340, 2240272, 1016517, 2240272,
       1345932,  539739, 2401062, 2314257, 1246094, 2505393, 2469339,
       2436067, 1711405,  772585, 2430911, 2138316, 1316794], dtype=uint32),
               random_state=RandomState(MT19937) at 0x18AB0F97E40)

In [7]:
import numpy as np
import os

def count_images_in_directory(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

test_directory = "C:/NewML/MachineLearning/env/archive/test/test"

# Load and preprocess test data
test_data = load_and_preprocess_images(test_directory, image_width, image_height)
print("Number of test images:", len(test_data))
# Count the number of weapon and no-weapon images in the test directory
num_test_weapon_images = count_images_in_directory(os.path.join(test_directory, 'weapon'))
num_test_no_weapon_images = count_images_in_directory(os.path.join(test_directory, 'no_weapon'))

# Convert the list of images into a NumPy array
test_data = np.array(test_data)

# Flatten the test data
flattened_test_data = test_data.reshape(len(test_data), -1)

# Use the trained HMM model to predict labels for the test data
predicted_labels = model.predict(flattened_test_data)

# Define the true labels for the test data (1 for weapon, 0 for no weapon)
true_labels = [1] * num_test_weapon_images + [0] * num_test_no_weapon_images

# Calculate accuracy
correct_predictions = sum(1 for true_label, predicted_label in zip(true_labels, predicted_labels) if true_label == predicted_label)
total_predictions = len(true_labels)
accuracy = correct_predictions / total_predictions

print("Accuracy: {:.2f}%".format(accuracy * 100))
test_perplexity = model.score(flattened_test_data)
print("Test Perplexity:", test_perplexity)

Number of test images: 289
Accuracy: 40.83%
Test Perplexity: -64139551.78855668
